In [32]:
import pandas as pd 
import datetime as dt

In [33]:
df = pd.read_csv("BigBasketDataset.tsv", sep="\t")

In [34]:
df.head()

,Member,Order,SKU,Created On,Description
0,M09736,6468572,34993740,22-09-2014 22:45,Other Sauces
1,M09736,6468572,15669800,22-09-2014 22:45,Cashews
2,M09736,6468572,34989501,22-09-2014 22:45,Other Dals
3,M09736,6468572,7572303,22-09-2014 22:45,Namkeen
4,M09736,6468572,15669856,22-09-2014 22:45,Sugar


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62141 entries, 0 to 62140
Data columns (total 5 columns):
Member         62141 non-null object
Order          62141 non-null int64
SKU            62141 non-null int64
Created On     62141 non-null object
Description    62141 non-null object
dtypes: int64(2), object(3)
memory usage: 2.4+ MB


In [36]:
print("Number of orders: ", df.Order.nunique())
print("Number of products bought: ",df.SKU.nunique())
print("Number of customers:", df.Member.nunique() )


Number of orders:  8387
Number of products bought:  1732
Number of customers: 106


## Creating RFQ - Dataframe

In [37]:
df_rfq=df.copy()



In [38]:
df_rfq["Created On"]=pd.to_datetime(df_rfq["Created On"])
df_rfq["Date"]=pd.DatetimeIndex(df_rfq["Created On"])

In [39]:
#Recency: Baseline data (last date in the dataset)
basedate=df_rfq.Date.max()

In [40]:
recency_df = df_rfq.groupby(by='Member', as_index=False)['Date'].max()
recency_df.columns = ['Member','LastOrder']

In [41]:
recency_df['Recency'] = recency_df['LastOrder'].apply(lambda x: (basedate - x).days)
recency_df.drop('LastOrder',axis=1,inplace=True)

In [42]:
#Frequency
df_temp=df_rfq.drop_duplicates(subset=['Order', 'Member'], keep="first")
#calculate frequency of orders
frequency_df = df_temp.groupby(by=['Member'], as_index=False)['Order'].count()

In [43]:
#Quantity
quantity_df=df_rfq.groupby(by=['Member'],as_index=False)['SKU'].count()

In [44]:
#Merge and rename
df_temp = recency_df.merge(frequency_df,on='Member')
df_rfq=df_temp.merge(quantity_df,on='Member')
df_rfq.rename(columns={'Order':'Frequency','SKU':'Quantity'},inplace=True)